In [1]:
import numpy as np
import pandas as pd

In [2]:
!kaggle datasets download -d "lakshmi25npathi/imdb-dataset-of-50k-movie-reviews"

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 97% 25.0M/25.7M [00:02<00:00, 21.8MB/s]
100% 25.7M/25.7M [00:02<00:00, 12.2MB/s]


In [3]:
!unzip /content/imdb-dataset-of-50k-movie-reviews.zip

Archive:  /content/imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


Using Transformers(BERT)

In [4]:
df = pd.read_csv('/content/IMDB Dataset.csv')

In [5]:
df['sentiment'] = df['sentiment'].replace(['negative', 'positive'], [0, 1])
X = df['review'].values
y = df['sentiment'].values

<ipython-input-5-aad2a5441ed9>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace(['negative', 'positive'], [0, 1])


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from transformers import TFAutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = TFAutoModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [8]:
def tokenize(batch):
    return tokenizer(batch, padding=True, truncation=True, max_length=128)

In [9]:
train_data_encoded = tokenizer(list(X_train), padding=True, truncation=True, return_tensors='tf', max_length=128)
test_data_encoded = tokenizer(list(X_test), padding=True, truncation=True, return_tensors='tf', max_length=128)

In [10]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': train_data_encoded['input_ids'],
     'attention_mask': train_data_encoded['attention_mask']},
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': test_data_encoded['input_ids'],
     'attention_mask': test_data_encoded['attention_mask']},
    y_test
))

In [11]:
BATCH_SIZE = 8
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [12]:
class BERTForClassification(tf.keras.Model):
    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]  # Extract pooled output
        return self.fc(x)

# Initialize the BERT classification model
classifier = BERTForClassification(model, num_classes=2)  # 2 classes: positive and negative


In [13]:
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [14]:
history = classifier.fit(
    train_dataset,
    epochs=2
)

Epoch 1/2
5000/5000 [==============================] - 1297s 251ms/step - loss: 0.3060 - accuracy: 0.8674
Epoch 2/2
5000/5000 [==============================] - 1250s 250ms/step - loss: 0.1902 - accuracy: 0.9250


In [15]:
# Evaluate the model
classifier.evaluate(test_dataset)

# Save the model if needed
classifier.save("sentiment_analysis_model")

1250/1250 [==============================] - 112s 88ms/step - loss: 0.2586 - accuracy: 0.8985


In [16]:

# Load IMDB dataset

# Preprocess the data (cleaning and splitting)

# Split the dataset into training and test sets

# Load the tokenizer and model from Hugging Face


# Tokenizing the dataset


# Tokenize the data


# Convert to TensorFlow Dataset




# Batch and shuffle the training dataset


# Custom BERT for Classification model

# Compile the model


# Train the model




In [19]:
# Assuming your model and tokenizer are already defined
def predict(input_text):
# Tokenize the new input data
  def preprocess_input(input_data):
      return tokenizer(input_data, padding=True, truncation=True, return_tensors='tf', max_length=128)

# For a single sentence
#input_text = ["I love this product!"]

# Tokenize the input text
  input_data_encoded = preprocess_input(input_text)

# Make predictions
  predictions = classifier.predict({
      'input_ids': input_data_encoded['input_ids'],
      'attention_mask': input_data_encoded['attention_mask']
  })

# Get predicted class (the highest probability class)
  predicted_class = tf.argmax(predictions, axis=-1).numpy()

# Map the predicted class to the actual label (e.g., 0 -> negative, 1 -> positive)
  labels = ["negative", "positive", "neutral"]  # Example labels for sentiment analysis
  predicted_label = labels[predicted_class[0]]

  print(f"Predicted label: {predicted_label}")


In [20]:
predict("worst movie i have ever wactched")

1/1 [==============================] - 5s 5s/step
Predicted label: negative


In [21]:
predict("I was extremely disappointed. Not what I expected at all")

1/1 [==============================] - 0s 83ms/step
Predicted label: negative


In [22]:
predict('I recently visited [Business Name] and was blown away by the level of customer service. The staff was incredibly attentive and made sure all my needs were met. The product I purchased exceeded my expectations in terms of quality and value.')

1/1 [==============================] - 0s 84ms/step
Predicted label: positive


In [23]:
predict('Fails to listen actively during conversations, leading to miscommunication and errors.')

1/1 [==============================] - 0s 105ms/step
Predicted label: negative
